# Training with Single batch, PyTorch CNN


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

In [2]:
class NetWork(nn.Module):
    
    def __init__(self):
        super(NetWork, self).__init__()
        
        #### conv layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)  
        
        #### Linear layers
        self.fc1 = nn.Linear(in_features= 12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120 , out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
            
        
    def forward(self, t):
        t = t #input layer
        
        #### hidden conv1 layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden conv2 layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #### hidden Linear_1 layer
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        #### hidden Linear_2 layer
        t = self.fc2(t)
        t = F.relu(t)
        
        #### hidden Linear_out layer
        t = self.out(t)
        
        return t
        

#### Fetch the data

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    './FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]) # converting ToTensor
)

#### Load data

In [4]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

#### Create a Batch

In [5]:
batch = next(iter(train_loader))
images, labels = batch

#### Initiate the Neural Network

In [6]:
network = NetWork()

## Calculate the loss

In [7]:
#### lets pass the images to the network and get the prediction
preds = network(images)

#### then we are going to access the cross_entropy loss func from the PyTorch  functional API
loss = F.cross_entropy(preds, labels)
print(loss, '\n', loss.item())

tensor(2.3097, grad_fn=<NllLossBackward0>) 
 2.309678077697754


# Calculate the Gradients

In [8]:
#### First lets check our original gradients value
print(network.conv1.weight.grad) # will return None

None


In [9]:
#### Now lets run our loss func when by using the back-propagation
loss.backward()

In [10]:
#### Now lets see the grad
print(network.conv1.weight.grad.shape) 

torch.Size([6, 1, 5, 5])


### Our gradient had been updated . 👆

# Now Use the Gradients & Update the NN weights

In [23]:
#### network.parameters() ----> are the weithts
optimizer = optim.Adam(network.parameters(), lr=0.01) 
loss.item()

2.2656915187835693

In [17]:
#### will calu the num of correct prediction out of the batch_size
def get_num_corrected(pred, labels):
    return pred.argmax(dim=1).eq(labels).sum().item()
get_num_corrected(preds, labels)    

4

In [22]:
compaireIT = preds.argmax(dim=1).eq(labels).sum().item()
compaireIT 

4

# Update the Weights

In [19]:
optimizer.step()  #### updating the weights of our NN

#### To illustrate this feed the NN the same images and check the loss

In [20]:
preds_2 = network(images)
loss = F.cross_entropy(preds_2, labels)
loss.item() # we will get less loss than before

2.2656915187835693

In [21]:
#### check the num of correct prediction
get_num_corrected(preds_2, labels)

11

# Training with Single batch, 
#### all the above cells in one cell


In [39]:
#### Train_loader and #### optimizer
train_loader_2 = torch.utils.data.DataLoader(train_set, batch_size=100)
loss_2 = optim.Adam(network.parameters(), lr=0.01) # network.parameters() --> are weights
print(loss_2)
#### batch
batch_2 = next(iter(train_loader))
images_2, labels_2 = batch_2

#### preds
preds_2 = network(images_2) # predi by passing the images_2

#### loss
loss_2 = F.cross_entropy(preds_2, labels_2) # calculate loss

#### loss-backwar
loss_2.backward() # calcu gradient

#### optimizer-step
optimizer.step() # update weight


print(f'loss: {loss_2.item()}')

preds_3 = network(images_2)
loss_3 = F.cross_entropy(preds_3, labels_2)

print(f'loss: {loss_3.item()}')

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
loss: 1.7236833572387695
loss: 1.5216926336288452
